<a href="https://colab.research.google.com/github/lephuocdat2000/-CS114.K21-/blob/master/SER/Eng_Vie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import scipy.io.wavfile
import numpy as np
import sys
import glob 
import librosa
import os
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import zscore
from IPython.display import Audio
### Time Distributed ConvNet imports ###
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, TimeDistributed, concatenate
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, BatchNormalization, LeakyReLU, Flatten
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import backend as K
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model
from sklearn.preprocessing import LabelEncoder
from IPython.display import Image
from glob import glob
import pickle
import itertools
from sklearn.model_selection import train_test_split
from PIL import Image

### Audioimport ###
import IPython

### Warning ###
import warnings
warnings.filterwarnings('ignore')

#Download dataset from Kaggle

In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/uwrfkaggler/ravdess-emotional-speech-audio")

#Extract signals and labels

In [3]:
sample_rate = 16000     
max_pad_len = 49100
eng_signals = []
eng_labels = []
dataset_path = '/content/drive/MyDrive/Nhận dạng/ravdess-emotional-speech-audio'
actors = os.listdir(dataset_path)

for actor_file in actors:
    actor_path = os.path.join(dataset_path,actor_file)
    filenames = os.listdir(actor_path)
    for name in filenames:
        temp = int(name[6:8])
        if (temp==1) or (temp==3) or (temp==4) or (temp==5):
            if temp==1:  eng_labels.append(3)
            elif temp==3: eng_labels.append(1)
            elif temp==4: eng_labels.append(2)
            else: eng_labels.append(0)
            file_path = os.path.join(actor_path,name)
            y,sr = librosa.load(file_path, sr=sample_rate)
            y = zscore(y)
            if len(y) < max_pad_len:    
                y_padded = np.zeros(max_pad_len)
                y_padded[:len(y)] = y
                y = y_padded
            elif len(y) > max_pad_len:
                y = np.asarray(y[:max_pad_len])
            eng_signals.append(y)

eng_labels = np.array(eng_labels)
eng_signals = np.array(eng_signals)

#Augment noisy

In [4]:
nb_augmented = 2

# Function to add noise to a signals with a desired Signal Noise ratio (SNR)
def noisy_signal(signal, snr_low=15, snr_high=30, nb_augmented=2):
    
    # Signal length
    signal_len = len(signal)

    # Generate White noise
    noise = np.random.normal(size=(nb_augmented, signal_len))
    
    # Compute signal and noise power
    s_power = np.sum((signal / (2.0 ** 15)) ** 2) / signal_len
    n_power = np.sum((noise / (2.0 ** 15)) ** 2, axis=1) / signal_len
    
    # Random SNR: Uniform [15, 30]
    snr = np.random.randint(snr_low, snr_high)
    
    # Compute K coeff for each noise
    K = np.sqrt((s_power / n_power) * 10 ** (- snr / 10))
    K = np.ones((signal_len, nb_augmented)) * K
    
    # Generate noisy signal
    return signal + K.T * noise

print("Data Augmentation: START")
eng_augmented_signals = list(map(noisy_signal, eng_signals))
print("Data Augmentation: END!")

Data Augmentation: START
Data Augmentation: END!


#Extract Mel

In [9]:
def mel_spectrogram(y, sr=16000, n_fft=512, win_length=256, hop_length=128, window='hamming', n_mels=128, fmax=4000):
    
    # Compute spectogram
    mel_spect = np.abs(librosa.stft(y, n_fft=n_fft, window=window, win_length=win_length, hop_length=hop_length)) ** 2
    
    # Compute mel spectrogram
    mel_spect = librosa.feature.melspectrogram(S=mel_spect, sr=sr, n_mels=n_mels, fmax=fmax)
    
    # Compute log-mel spectrogram
    mel_spect = librosa.power_to_db(mel_spect, ref=np.max)
    
    return mel_spect


# Start feature extraction
print("Feature extraction: START")

# Compute spectogram for all audio file
eng_mel_spects = np.asarray(list(map(mel_spectrogram, eng_signals)))
eng_augmented_mel_spects = [np.asarray(list(map(mel_spectrogram, eng_augmented_signals[i]))) for i in range(len(eng_augmented_signals))]

# Stop feature extraction
print("Feature extraction: END!")

Feature extraction: START
Feature extraction: END!


#Prepare Data

Training set

In [ ]:
# Build augmented labels and train
eng_aug_labels = np.asarray(list(itertools.chain.from_iterable([[label] * nb_augmented for label in eng_labels])))
eng_AUG_MEL_SPECTs = np.asarray(list(itertools.chain.from_iterable(eng_augmented_mel_spects)))
eng_X_train = np.concatenate((eng_mel_spects,eng_AUG_MEL_SPECTs))
eng_y_train = np.concatenate((eng_labels,eng_aug_labels))

Test set